In [3]:
import uuid
import random
import json
from confluent_kafka import Producer
import time

from ejercicios.pokemons import SEED, TOPIC_USERS

user_names = [
    'Luis',
    'Jose',
    'Maria',
    'Laura',
    'Fran',
    'Dani',
    'David',
    'Cris',
    'Oscar',
    'Virginia'
]

users = {}
for u in range(5):
    i = str(uuid.uuid4())
    name = user_names[u % len(user_names)]
    age = random.randint(18, 99)
    users[i] = ({'id': i, 'name': name, 'age': age, 'logged': False})

In [30]:
def choose_partition(num_partitions, key):
    if num_partitions == 1 or key is None:
        return 0
    else:
        try:
            as_num = int(key[0], base=16)
            return as_num % num_partitions
        except:
            return 0

print(choose_partition(1, str(uuid.uuid4())))
print(choose_partition(1, None))

for i in range(10):
    u = str(uuid.uuid4())
    print("{}: {}".format(u, choose_partition(2, u)))

print()
for i in range(10):
    u = str(uuid.uuid4())
    print("{}: {}".format(u, choose_partition(3, u)))


0
0
cd7fc96c-d459-4764-a02f-0d593f09956c: 0
25125163-f794-4a05-aa02-1a4bb25bb8a4: 0
c3e5ee4d-6a4b-4d9d-a86b-f6a2f1e1fea4: 0
d4a4236c-eb53-4bf6-8839-d07653098be1: 1
1b8bed8a-b447-49c6-b9d3-24141730437c: 1
eb9624a8-6ce8-416d-938f-b81d8718dfad: 0
ee1783c5-2966-4aac-9201-cb67f5adafcb: 0
1a526524-fb02-49ab-b244-011b373c7e59: 1
9c591811-4473-4f41-87c6-8d5b8b60431b: 1
b926828e-7a7d-4932-a38d-0b1eca3e0412: 1

f5cf9de2-3c6c-4668-82c9-ef9edbef3e16: 0
44564bc7-bf9f-4334-8836-94e77e3ac6f5: 1
b257a7ed-a876-46d5-8290-fc19046410ae: 2
aa9eec6e-e3ab-459c-aba5-5b2cba8afd9d: 1
4a1ba9ac-e8d9-47fd-a88b-8764c1d2dc72: 1
0b4b5859-d426-4146-b4d7-284ae5e5b89b: 0
ff065d60-122b-4976-8b75-7f4b28cb98f5: 0
3de0438f-c2c8-4fe3-9a15-985406f91e75: 0
55a3f526-1856-4743-abed-70157b4ddee0: 2
1ebd4bd2-2291-4c8a-a4d5-939671d37190: 1


In [19]:
a = int('d', base=16)
a % 3

1

In [14]:
15 % 2

1

In [32]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

while True:
    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)
    topic_list = p.list_topics(TOPIC_USERS)
    topic_metadata = topic_list.topics[TOPIC_USERS]
    num_partitions = len(topic_metadata.partitions.keys())
    
    user_id = random.choice(list(users.keys()))
    if users[user_id]['logged']:
        print("Logging {} OFF".format(user_id))
        users[user_id]['logged'] = False
        p.produce(TOPIC_USERS,
                  key=user_id,
                  partition=choose_partition(num_partitions, user_id),
                  value=None,
                  callback=delivery_report)
    else:
        print("Logging {} IN".format(user_id))
        users[user_id]['logged'] = True
        p.produce(TOPIC_USERS,
                  key=user_id,
                  partition=choose_partition(num_partitions, user_id),
                  value=json.dumps(users[user_id]).encode('utf-8'),
                  callback=delivery_report)

    time.sleep(abs(random.gauss(5, 3)))

# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()


Logging 0dc09f77-9d48-48ec-92e8-85b2c048550c IN
Message delivered to users [0]
Logging 658880d7-9622-4254-8dd2-b733eb63c806 OFF
Message delivered to users [0]
Logging 90f02947-4ff3-41cc-81d9-1459613c4984 OFF
Message delivered to users [1]
Logging 537fceb3-3fb0-4545-94f7-c915aee3ab1d IN
Message delivered to users [1]
Logging 658880d7-9622-4254-8dd2-b733eb63c806 IN
Message delivered to users [0]
Logging 0dc09f77-9d48-48ec-92e8-85b2c048550c OFF
Message delivered to users [0]
Logging 46f62537-18dc-40ff-807b-070baebf1991 IN
Message delivered to users [0]
Logging 90f02947-4ff3-41cc-81d9-1459613c4984 IN
Message delivered to users [1]
Logging 0dc09f77-9d48-48ec-92e8-85b2c048550c IN
Message delivered to users [0]
Logging 537fceb3-3fb0-4545-94f7-c915aee3ab1d OFF
Message delivered to users [1]
Logging 0dc09f77-9d48-48ec-92e8-85b2c048550c OFF
Message delivered to users [0]
Logging 537fceb3-3fb0-4545-94f7-c915aee3ab1d IN
Message delivered to users [1]
Logging 90f02947-4ff3-41cc-81d9-1459613c4984 OF

KeyboardInterrupt: 